In [1]:
import os,sys

In [15]:
import pandas as pd
df=pd.DataFrame(columns=["data","label"],data=[["a","b"],[1,2]])
df

,data,label
0,a,b
1,1,2


In [12]:
joblib.dump()

<function joblib.numpy_pickle.dump(value, filename, compress=0, protocol=None, cache_size=None)>

In [8]:
import os,sys
import pandas as pd
data=[]
label=[]
data_path="data/"
for path in os.listdir(data_path):
    for text_path in os.listdir(data_path+path):
        data.append(open(data_path+path+"/"+text_path,encoding='utf-8').read())
        label.append(path)
df=pd.DataFrame({"data":data,"label":label})
df.to_excel("data.xlsx",index=False)

In [12]:
import os,sys
import pandas as pd
data=[]
label=[]
data_path="ocrs/"
for path in os.listdir(data_path):
    for text_path in os.listdir(data_path+path):
        data.append(open(data_path+path+"/"+text_path,encoding='utf-8').read())
        label.append(path)
df=pd.DataFrame({"data":data,"label":label})
df.to_excel("newdata.xlsx",index=False)

In [30]:
data_path+path+"/"+text_path

'data/food/food_58.txt'

In [9]:
import pandas as pd
df=pd.read_excel(read_file_io("Classification/Dataset/data.xlsx"),dtype=str)
df

,data,label
0,Lufthansa flies back to profit\n\nGerman airli...,business
1,Winn-Dixie files for bankruptcy\n\nUS supermar...,business
2,US economy still growing says Fed\n\nMost area...,business
3,Saab to build Cadillacs in Sweden\n\nGeneral M...,business
4,Bank voted 8-1 for no rate change\n\nThe decis...,business
...,...,...
995,Mobile games come of age\n\nThe BBC News websi...,technologie
996,California sets fines for spyware\n\nThe maker...,technologie
997,Web helps collect aid donations\n\nThe web is ...,technologie
998,Mobiles rack up 20 years of use\n\nMobile phon...,technologie


In [42]:
df.to_excel("data.xlsx",index=False)

In [4]:
import pandas as pd
df=pd.read_excel("data.xlsx",dtype=str)
df

,data,label
0,Lufthansa flies back to profit\n\nGerman airli...,business
1,Winn-Dixie files for bankruptcy\n\nUS supermar...,business
2,US economy still growing says Fed\n\nMost area...,business
3,Saab to build Cadillacs in Sweden\n\nGeneral M...,business
4,Bank voted 8-1 for no rate change\n\nThe decis...,business
...,...,...
995,Mobile games come of age\n\nThe BBC News websi...,technologie
996,California sets fines for spyware\n\nThe maker...,technologie
997,Web helps collect aid donations\n\nThe web is ...,technologie
998,Mobiles rack up 20 years of use\n\nMobile phon...,technologie


In [10]:
df.isnull().sum()

data     0
label    0
dtype: int64

# Training the Model

In [6]:
import os,io,json
from difflib import get_close_matches
from google.cloud import storage
#from secret import get_service_account
from google.oauth2 import service_account
#os.environ['GOOGLE_APPLICATION_CREDENTIALS']=os.environ['gcp']
if 'gcp_secret' in os.environ and os.environ['gcp_secret'] is not None:
    os.environ['GOOGLE_APPLICATION_CREDENTIALS']=os.environ['gcp']
storage_client = storage.Client()
bucket_name=os.environ['bucket_name']
bucket = storage_client.bucket(bucket_name)
from google.cloud import vision
import time
import io
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
def read_file(filename):
    blob=bucket.blob(filename)
    return blob.download_as_string()
def read_file_io(filename):
    blob=bucket.blob(filename)
    f=io.BytesIO()
    blob.download_to_file(f)
    f.seek(0)
    return f
def write_file_io(filename,content):
    blob=bucket.blob(filename)
    blob.upload_from_file(content)
    return 'completed'
def write_file(filename,content):
    blob=bucket.blob(filename)
    blob.upload_from_string(content)
    return 'completed'
def list_blob_all(prefix=None):
    blobs = storage_client.list_blobs(os.environ['bucket_name'],prefix=prefix)
    for blob in blobs:
        print(blob.name)
def download_to_local(filename,name):
    blob=bucket.blob(filename)
    blob.download_to_filename(name)

In [1]:
import time,os,sys,re
import numpy as np
import pandas as pd
import pickle
import nltk
st=time.time()
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer,word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score
import joblib
stop = stopwords.words('english')
def clean(text):
    word_lem=WordNetLemmatizer()
    tokens=word_tokenize(text)
    lower=[word.lower() for word in tokens if len(word)>2 and word.isalpha()]
    lemmatized_text=[word_lem.lemmatize(word) for word in lower]
    return lemmatized_text
def vectorize(data,tfidf_vect_fit):
    x_tfidf=tfidf_vect_fit.transform(data)
    x_tfidf_df=pd.DataFrame(x_tfidf.todense(),columns=tfidf_vect_fit.get_feature_names())
    return x_tfidf_df
import pandas as pd
df=pd.read_excel("data.xlsx",dtype=str)
df.dropna(subset=["data","label"],inplace=True)
X_train,X_test,Y_train,Y_test=train_test_split(df["data"],df["label"],test_size=0.1,random_state=42)
tfidf_vect=TfidfVectorizer(analyzer=clean,use_idf=True,max_features=1000)
tfidf_vect_fit=tfidf_vect.fit(X_train.values.astype('U'))
joblib.dump(tfidf_vect_fit,"tfidf.pkl")
X_train=vectorize(X_train.values.astype('U'),tfidf_vect_fit)
model=RandomForestClassifier(n_jobs=-1,n_estimators=100)
model.fit(X_train,Y_train)
joblib.dump(model,"model.pkl",compress=3)
y_pred=model.predict(vectorize(X_test,tfidf_vect_fit))
print("Accuracy",round(accuracy_score(Y_test,y_pred),3))
print(time.time()-st)

C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance:

C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current

C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current

Accuracy 0.92
13.537550687789917


C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


In [2]:
loss
loss

,ability,able,about,above,access,according,across,act,action,activity,...,would,writes,writing,wrote,year,yet,york,you,young,your
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.0,0.030159,0.016949,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.253360,0.000000,0.257211
2,0.0,0.040640,0.000000,0.000000,0.197981,0.000000,0.0,0.000000,0.000000,0.0,...,0.093550,0.0,0.0,0.0,0.022436,0.042755,0.0,0.000000,0.000000,0.000000
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.052016,0.000000,0.068648
4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.096284,0.082875,0.0,...,0.090992,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,0.0,0.000000,0.034796,0.000000,0.000000,0.060034,0.0,0.075406,0.000000,0.0,...,0.035631,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
896,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.141907,0.000000,0.187283
897,0.0,0.000000,0.032330,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.031759,0.000000,0.0,0.037175,0.000000,0.000000
898,0.0,0.015078,0.016947,0.018876,0.000000,0.000000,0.0,0.018363,0.031611,0.0,...,0.078091,0.0,0.0,0.0,0.000000,0.015862,0.0,0.000000,0.000000,0.000000


In [3]:
clean("hii ** 123")

['hii']

In [39]:
st=time.time()
print(read_file("Classification/Models/status.txt").decode()=='1')
print(time.time()-st)

NameError: name 'read_file' is not defined

In [13]:
def fun(a):
    return a

In [40]:
import pandas as pd
df=pd.read_excel("data.xlsx",dtype=str)
df.dropna(subset=["data","label"],inplace=True)

# Testing the Model

In [4]:
import time,os,sys,re
import numpy as np
import pandas as pd
import pickle
import nltk
st=time.time()
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer,word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score
import joblib
def clean(text):
    word_lem=WordNetLemmatizer()
    tokens=word_tokenize(text)
    lower=[word.lower() for word in tokens if len(word)>2 and word.isalpha()]
    lemmatized_text=[word_lem.lemmatize(word) for word in lower]
    return lemmatized_text
def vectorize(data,tfidf_vect_fit):
    x_tfidf=tfidf_vect_fit.transform(data)
    x_tfidf_df=pd.DataFrame(x_tfidf.todense(),columns=tfidf_vect_fit.get_feature_names())
    return x_tfidf_df
import joblib
model=joblib.load("model.pkl")
tfidf_vect_fit=joblib.load("tfidf.pkl")
ocr_data=df["data"][620]
df2=pd.DataFrame({"data":[ocr_data]},dtype=str)
pred_value=model.predict(vectorize(df2["data"].values.astype('U'),tfidf_vect_fit))
pred_value[0]

C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


'politics'

In [5]:
ocr_data

'Blair returns from peace mission\n\nPrime Minister Tony Blair has arrived back from his diplomatic mission to the Middle East to try to resurrect the peace process.\n\nMr Blair held talks with his Israeli counterpart, Ariel Sharon, and the Palestinian leader, Mahmoud Abbas. He confirmed that a renewed drive to reform the Palestinian Authority and address security issues would come at a London conference in March. Mr Blair also made a surprise trip to Iraq this week. The Israelis described the meeting as important but said they would not need to attend. Mr Blair briefly visited the tomb of Yasser Arafat in Ramallah - the first world leader to do so. He nodded briefly towards the tomb, rather than lay a wreath, in what Palestinian officials said was a compromise gesture agreed at the last minute.\n\nThe BBC\'s Paul Reynolds says the London conference will be a limited measure to shore up the leadership of Mahmoud Abbas, also known as Abu Mazen, who is expected to win the Palestinian pre

In [42]:
model.predict_proba(vectorize(df2["data"].values.astype('U'),tfidf_vect_fit)).max()

0.871

In [37]:
import multiprocessing
multiprocessing.cpu_count()

8

In [23]:
import forestci
a=random_forest_error(model, vectorize(df2["data"].values.astype('U'),tfidf_vect_fit), 'sport', inbag=None, calibrate=True, memory_constrained=False, memory_limit=None)

ModuleNotFoundError: No module named 'forestci'

In [18]:
!pip freeze

absl-py==0.12.0
aiohttp==3.8.1
aiosignal==1.2.0
alabaster==0.7.12
altgraph==0.17
anaconda-client==1.7.2
anaconda-navigator==1.9.7
anaconda-project==0.8.3
anyascii==0.3.1
anyio==3.6.1
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asgiref==3.2.3
asn1crypto==1.0.1
astroid==2.3.1
astropy==3.2.1
astunparse==1.6.3
async-timeout==4.0.2
asynctest==0.13.0
atomicwrites==1.3.0
attrs==19.2.0
auto-py-to-exe==2.6.6
Babel==2.7.0
backcall==0.1.0
backports.functools-lru-cache==1.5
backports.os==0.1.1
backports.shutil-get-terminal-size==1.0.0
backports.tempfile==1.0
backports.weakref==1.0.post1
beautifulsoup4==4.8.0
bitarray==1.0.1
bkcharts==0.2
bleach==3.1.0
bokeh==1.3.4
boto==2.49.0
bottle==0.12.18
bottle-websocket==0.2.9
Bottleneck==1.2.1
branca==0.3.1
cachetools==4.2.1
certifi==2019.9.11
cffi==1.12.3
chardet==3.0.4
charset-normalizer==2.1.1
Click==7.0
cloudpickle==1.2.2
clyent==1.2.2
colorama==0.4.1
comtypes==1.1.7
conda==4.7.12
conda-build==3.18.9
conda-package-handling==1.6.0
conda-verify==3.4.

In [84]:
df.dropna(subset=["data","label"],inplace=True)
X_train,X_test,Y_train,Y_test=train_test_split(df["data"],df["label"],test_size=0.1,random_state=2)
tfidf_vect=TfidfVectorizer(analyzer=clean,use_idf=True,max_features=1000)
tfidf_vect_fit=tfidf_vect.fit(X_train.values.astype('U'))
joblib.dump(tfidf_vect_fit,"tfidf.pkl")
X_train=vectorize(X_train.values.astype('U'),tfidf_vect_fit)
model=RandomForestClassifier(n_jobs=-1)
model.fit(X_train,Y_train)
y_pred=model.predict(vectorize(X_test,tfidf_vect_fit))
print("Accuracy",round(accuracy_score(Y_test,y_pred),3))

C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy 0.85


In [41]:
df.dropna(subset=["data","label"],inplace=True)
X_train,X_test,Y_train,Y_test=train_test_split(df["data"],df["label"],test_size=0.1,random_state=2)

In [68]:
tfidf_vect=TfidfVectorizer(analyzer=clean,use_idf=True,max_features=1000)
tfidf_vect_fit=tfidf_vect.fit(X_train.values.astype('U'))
joblib.dump(tfidf_vect_fit,"tfidf.pkl")

['tfidf.pkl']

In [65]:
b=tfidf_vect_fit.get_feature_names()

In [73]:
X_train=vectorize(X_train.values.astype('U'),tfidf_vect_fit)

In [74]:
model=RandomForestClassifier(n_jobs=-1)

In [76]:
model.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [77]:
y_pred=model.predict(vectorize(X_test,tfidf_vect_fit))

In [78]:
y_pred

array(['business', 'space', 'sport', 'food', 'technologie', 'politics',
       'historical', 'food', 'sport', 'food', 'entertainment',
       'technologie', 'medical', 'space', 'sport', 'politics', 'space',
       'politics', 'historical', 'business', 'food', 'business', 'food',
       'sport', 'entertainment', 'politics', 'graphics', 'entertainment',
       'business', 'politics', 'politics', 'food', 'food', 'space',
       'historical', 'entertainment', 'entertainment', 'technologie',
       'medical', 'politics', 'politics', 'technologie', 'sport',
       'medical', 'medical', 'graphics', 'food', 'sport', 'space',
       'technologie', 'business', 'space', 'historical', 'graphics',
       'sport', 'technologie', 'medical', 'medical', 'entertainment',
       'sport', 'entertainment', 'technologie', 'entertainment',
       'historical', 'entertainment', 'business', 'politics', 'medical',
       'graphics', 'graphics', 'food', 'food', 'food', 'technologie',
       'food', 'space', 'gra

In [81]:
round(accuracy_score(Y_test,y_pred),3)

0.88

In [97]:
import time,os,sys,re
import numpy as np
import pandas as pd
import pickle
import nltk
st=time.time()
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer,word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score
import joblib
stop = stopwords.words('english')
from nltk.corpus import wordnet
def clean(text):
    word_lem=WordNetLemmatizer()
    tokens=word_tokenize(text)
    lower=[word.lower() for word in tokens if len(word)>2 and word.isalpha()]
    lemmatized_text=[word_lem.lemmatize(word) for word in lower]
    return lemmatized_text
def vectorize(data,tfidf_vect_fit):
    x_tfidf=tfidf_vect_fit.transform(data)
    x_tfidf_df=pd.DataFrame(x_tfidf.todense(),columns=tfidf_vect_fit.get_feature_names())
    return x_tfidf_df
import pandas as pd
df=pd.read_excel("data.xlsx",dtype=str)
df.dropna(subset=["data","label"],inplace=True)
X_train,X_test,Y_train,Y_test=train_test_split(df["data"],df["label"],test_size=0.1,random_state=42)
tfidf_vect=TfidfVectorizer(analyzer=clean,use_idf=True,max_features=1000)
tfidf_vect2=TfidfVectorizer(analyzer=dataProcess,use_idf=True,max_features=1000)
tfidf_vect_fit=tfidf_vect.fit(X_train.values.astype('U'))
tfidf_vect_fit2=tfidf_vect2.fit(X_train.values.astype('U'))
joblib.dump(tfidf_vect_fit,"tfidf.pkl")
X_train2=vectorize(X_train.values.astype('U'),tfidf_vect_fit2)
X_train=vectorize(X_train.values.astype('U'),tfidf_vect_fit)
model=RandomForestClassifier(n_jobs=-1)
model.fit(X_train,Y_train)
model2=RandomForestClassifier(n_jobs=-1)
model2.fit(X_train2,Y_train)
joblib.dump(model,"model.pkl",compress=3)
y_pred=model.predict(vectorize(X_test,tfidf_vect_fit))
y_pred2=model2.predict(vectorize(X_test,tfidf_vect_fit2))
print("Accuracy",round(accuracy_score(Y_test,y_pred),3))
print("Accuracy2yt",round(accuracy_score(Y_test,y_pred2),3))
print(time.time()-st)

C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy 0.86
Accuracy2yt 0.84
37.44888973236084


In [96]:
joblib.dump(tfidf_vect_fit2,"tfidf2.pkl")
joblib.dump(model2,"model2.pkl",compress=3)

['model2.pkl']

In [92]:
from nltk.corpus import wordnet
def remove_punctuation(text):
    import os,string,re,json
    text = ''.join([c if c not in string.punctuation else " " for c in text])
    return text
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return "n"
def dataProcess(data):
    #stopwords=stopwords.words('english')
    data=data.lower()
    filtered_sentence=remove_punctuation(data)
    word_tokens = word_tokenize(filtered_sentence)
    lower=[word.lower() for word in word_tokens if len(word)>2 and word.isalpha()]
    filtered_sentence = [w for w in lower if not w in stop] 
    lemmatizer = WordNetLemmatizer()
    pos_tagged = nltk.pos_tag(filtered_sentence)
    data=[lemmatizer.lemmatize(word,pos=pos_tagger(pos)) for word,pos in pos_tagged]
    final_data=" ".join(data)
    return data

In [93]:
dataProcess(ocr_data)

['bekele',
 'set',
 'sight',
 'world',
 'mark',
 'olympic',
 'champion',
 'kenenisa',
 'bekele',
 'determine',
 'add',
 'world',
 'indoor',
 'two',
 'mile',
 'record',
 'february',
 'norwich',
 'union',
 'grand',
 'prix',
 'birmingham',
 'year',
 'old',
 'chasing',
 'record',
 'hold',
 'compatriot',
 'mentor',
 'haile',
 'gebrselassie',
 'set',
 'mark',
 'meeting',
 'still',
 'hungry',
 'much',
 'sport',
 'say',
 'bekele',
 'aim',
 'two',
 'mile',
 'world',
 'record',
 'birmingham',
 'next',
 'target',
 'gebrselassie',
 'current',
 'record',
 'stand',
 'eight',
 'minute',
 'second',
 'bekele',
 'stranger',
 'overhaul',
 'world',
 'mark',
 'national',
 'indoor',
 'arena',
 'ethiopian',
 'break',
 'world',
 'indoor',
 'record',
 'debut',
 'meeting',
 'last',
 'year',
 'compatriot',
 'mulugeta',
 'wondimu',
 'abiyote',
 'abate',
 'markos',
 'geneti',
 'world',
 'indoor',
 'bronze',
 'medallist',
 'race',
 'bekele',
 'february',
 'meet',
 'already',
 'attract',
 'crop',
 'olympic',
 'talen

In [64]:
def async_detect_document(gcs_source_uri, gcs_destination_uri):
    """OCR with PDF/TIFF as source files on GCS"""
    import json
    import re
    from google.cloud import vision
    from google.cloud import storage

    # Supported mime_types are: 'application/pdf' and 'image/tiff'
    mime_type = 'application/pdf'

    # How many pages should be grouped into each json output file.
    batch_size = 2

    client = vision.ImageAnnotatorClient()

    feature = vision.Feature(
        type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)

    gcs_source = vision.GcsSource(uri=gcs_source_uri)
    input_config = vision.InputConfig(
        gcs_source=gcs_source, mime_type=mime_type)

    gcs_destination = vision.GcsDestination(uri=gcs_destination_uri)
    output_config = vision.OutputConfig(
        gcs_destination=gcs_destination)

    async_request = vision.AsyncAnnotateFileRequest(
        features=[feature], input_config=input_config,
        output_config=output_config)

    operation = client.async_batch_annotate_files(
        requests=[async_request])

    print('Waiting for the operation to finish.')
    operation.result(timeout=420)

    # Once the request has completed and the output has been
    # written to GCS, we can list all the output files.
    storage_client = storage.Client()

    match = re.match(r'gs://([^/]+)/(.+)', gcs_destination_uri)
    bucket_name = match.group(1)
    prefix = match.group(2)

    bucket = storage_client.get_bucket(bucket_name)

    # List objects with the given prefix, filtering out folders.
    blob_list = [blob for blob in list(bucket.list_blobs(
        prefix=prefix)) if not blob.name.endswith('/')]
    print('Output files:')
    for blob in blob_list:
        print(blob.name)

    # Process the first output file from GCS.
    # Since we specified batch_size=2, the first response contains
    # the first two pages of the input file.
    output = blob_list[0]

    json_string = output.download_as_string()
    response = json.loads(json_string)

    # The actual response for the first page of the input file.
    first_page_response = response['responses'][0]
    annotation = first_page_response['fullTextAnnotation']

    # Here we print the full text from the first page.
    # The response contains more information:
    # annotation/pages/blocks/paragraphs/words/symbols
    # including confidence scores and bounding boxes
    print('Full text:\n')
    print(annotation['text'])
    return response

In [73]:
import time
st=time.time()
res=async_detect_document("gs://noted-cortex-358103_research/Contract/Residential-Lease-Agreement-2.pdf","gs://noted-cortex-358103_research/Classification/Demo/1.ocr")
print(time.time()-st)

Waiting for the operation to finish.
Output files:
Classification/Demo/1.ocroutput-1-to-11.json
Full text:

RESIDENTIAL LEASE AGREEMENT
I. THE PARTIES. This Residential Lease Agreement (“Agreement”) made this
20 is between:
Landlord:
with a mailing address of
City of
("Landlord"), AND
State of
Tenant(s):
("Tenant").
Landlord and Tenant are each referred to herein as a "Party" and, collectively, as the
"Parties."
NOW, THEREFORE, FOR AND IN CONSIDERATION of the mutual promises and
agreements contained herein, the Tenant agrees to lease the Premises from the
Landlord under the following terms and conditions:
II. LEASE TYPE. This Agreement shall be considered a: (check one)
- Fixed Lease. The Tenant shall be allowed to occupy the Premises starting on
20 and end on
20 ("Lease
Term"). At the end of the Lease Term and no renewal is made, the Tenant:
(check one)
☐ - May continue to lease the Premises under the same terms of this
Agreement under a month-to-month arrangement.
- Must vacate the P

In [ ]:
def ocr_maker_1(pageinfo):
    rubber_main=pageinfo[1]
    pageinfo=pageinfo[0]
    print("pageinfo",pageinfo)
    bucket_name=os.environ['bucket_name']
    client = vision.ImageAnnotatorClient()
    print(client)
    st=time.time()
    image = vision.Image()
    value=pageinfo
    image.source.image_uri="gs://"+bucket_name+"/"+value
    print(value)
    #print(image)
    # Performs label detection on the image file
    response = client.document_text_detection(image=image)
    print(time.time()-st)
    # print(response)
    ocr=[]
    from collections import defaultdict
    rubber=defaultdict(list)
    documents=response.full_text_annotation
    for page in documents.pages:
        for block in page.blocks:
            for para in block.paragraphs:
                for word in para.words:
                    wor=""
                    for symbol in word.symbols:
                        wor+=symbol.text
                        
                        if (symbol.property.detected_break.type==1 or symbol.property.detected_break.type==3):
                            wor+=" "
                        if (symbol.property.detected_break.type==2):
                            wor+="\t"
                        if (symbol.property.detected_break.type==5):
                            wor+="\n"
                    ocr.append(wor)
                    rubber["1"].append([wor.strip(),word.bounding_box.vertices[0].x,word.bounding_box.vertices[2].x,word.bounding_box.vertices[0].y,word.bounding_box.vertices[2].y])
                    #print(symbol)
        rubber["img_H"].append(page.height)
        rubber["img_W"].append(page.width)
    #rubber_main[path]=rubber
    write_file(value+'_ocr.txt',"".join(ocr))
    rubber_main[pageinfo.split("/")[-1]]=rubber
    print('ocr_completed',pageinfo)
    del pageinfo,ocr,rubber

In [28]:
from google.cloud import vision
gcs_source_uri="gs://noted-cortex-358103_research/Contract/Residential-Lease-Agreement-2.pdf"

In [29]:
mime_type = 'application/pdf'
gcs_source = vision.GcsSource(uri=gcs_source_uri)
input_config = vision.InputConfig(
        gcs_source=gcs_source, mime_type=mime_type)

In [46]:
client = vision.ImageAnnotatorClient()
feature = vision.Feature(
        type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)
gcs_destination = vision.GcsDestination(uri=None)
output_config = vision.OutputConfig(
        gcs_destination=gcs_destination)
async_request = vision.AsyncAnnotateFileRequest(
        features=[feature], input_config=input_config,
        output_config=output_config)

In [49]:
operation = client.batch_annotate_files()

InvalidArgument: 400 Right now only one AnnotateFileRequest in BatchAnnotateFilesRequest is supported.

In [48]:
dir(client)

['DEFAULT_ENDPOINT',
 'DEFAULT_MTLS_ENDPOINT',
 'Feature',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_all_features',
 '_get_default_mtls_endpoint',
 '_transport',
 'annotate_image',
 'async_batch_annotate_files',
 'async_batch_annotate_images',
 'batch_annotate_files',
 'batch_annotate_images',
 'common_billing_account_path',
 'common_folder_path',
 'common_location_path',
 'common_organization_path',
 'common_project_path',
 'crop_hints',
 'document_text_detection',
 'face_detection',
 'from_service_account_file',
 'from_service_account_info',
 'from_service_account_json',
 'get_mtls_endpoint_and_cert_source',
 'image_properties',

In [51]:
from google.cloud import vision_v1
client = vision_v1.ImageAnnotatorClient()

# Supported mime_type: application/pdf, image/tiff, image/gif
mime_type = "application/pdf"
content=read_file_io("Contract/Residential-Lease-Agreement-2.pdf").read()
input_config = {"mime_type": mime_type, "content": content}
features = [{"type_": vision_v1.Feature.Type.DOCUMENT_TEXT_DETECTION}]

# The service can process up to 5 pages per document file. Here we specify
# the first, second, and last page of the document to be processed.
requests = [{"input_config": input_config, "features": features, "pages": []}]

response = client.batch_annotate_files(requests=requests)

In [63]:
for image_response in response.responses[0].responses:
        print(u"Full text: {}".format(image_response.full_text_annotation.text))

Full text: RESIDENTIAL LEASE AGREEMENT
I. THE PARTIES. This Residential Lease Agreement (“Agreement”) made this
20 is between:
Landlord:
with a mailing address of
City of
("Landlord"), AND
State of
Tenant(s):
("Tenant").
Landlord and Tenant are each referred to herein as a "Party" and, collectively, as the
"Parties."
NOW, THEREFORE, FOR AND IN CONSIDERATION of the mutual promises and
agreements contained herein, the Tenant agrees to lease the Premises from the
Landlord under the following terms and conditions:
II. LEASE TYPE. This Agreement shall be considered a: (check one)
- Fixed Lease. The Tenant shall be allowed to occupy the Premises starting on
20 and end on
20 ("Lease
Term"). At the end of the Lease Term and no renewal is made, the Tenant:
(check one)
☐ - May continue to lease the Premises under the same terms of this
Agreement under a month-to-month arrangement.
- Must vacate the Premises.
e
"
☐ - Month-to-Month Lease. The Tenant shall be allowed to occupy the Premises
on a mo

In [85]:
for i in res['responses']:
    print(i['fullTextAnnotation'])

{'pages': [{'property': {'detectedLanguages': [{'languageCode': 'en', 'confidence': 0.9803086}]}, 'width': 612, 'height': 792, 'blocks': [{'boundingBox': {'normalizedVertices': [{'x': 0.11764706, 'y': 0.06818182}, {'x': 0.7908497, 'y': 0.06818182}, {'x': 0.7908497, 'y': 0.14646465}, {'x': 0.11764706, 'y': 0.14646465}]}, 'paragraphs': [{'boundingBox': {'normalizedVertices': [{'x': 0.251634, 'y': 0.06818182}, {'x': 0.748366, 'y': 0.06818182}, {'x': 0.748366, 'y': 0.08585858}, {'x': 0.251634, 'y': 0.08585858}]}, 'words': [{'property': {'detectedLanguages': [{'languageCode': 'en', 'confidence': 1}]}, 'boundingBox': {'normalizedVertices': [{'x': 0.251634, 'y': 0.06818182}, {'x': 0.44444445, 'y': 0.06818182}, {'x': 0.44444445, 'y': 0.08585858}, {'x': 0.251634, 'y': 0.08585858}]}, 'symbols': [{'text': 'R', 'confidence': 0.95097744}, {'text': 'E', 'confidence': 0.95377606}, {'text': 'S', 'confidence': 0.92038316}, {'text': 'I', 'confidence': 0.95204604}, {'text': 'D', 'confidence': 0.9307687},

#new classification

In [90]:
import time,os,sys,re
import numpy as np
import pandas as pd
import pickle
import nltk
st=time.time()
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer,word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score
import joblib
stop = stopwords.words('english')
def clean(text):
    word_lem=WordNetLemmatizer()
    tokens=word_tokenize(text)
    lower=[word.lower() for word in tokens if len(word)>2 and word.isalpha() and word not in stop]
    lemmatized_text=[word_lem.lemmatize(word) for word in lower]
    return lemmatized_text
def vectorize(data,tfidf_vect_fit):
    x_tfidf=tfidf_vect_fit.transform(data)
    x_tfidf_df=pd.DataFrame(x_tfidf.todense(),columns=tfidf_vect_fit.get_feature_names())
    return x_tfidf_df
import pandas as pd
df=pd.read_excel("newdata.xlsx",dtype=str)
df.dropna(subset=["data","label"],inplace=True)
X_train,X_test,Y_train,Y_test=train_test_split(df["data"],df["label"],test_size=0.01)
tfidf_vect=TfidfVectorizer(analyzer=clean,use_idf=True,max_features=500)
tfidf_vect_fit=tfidf_vect.fit(X_train.values.astype('U'))
print(tfidf_vect_fit.get_feature_names())
joblib.dump(tfidf_vect_fit,"tfidf.pkl")
X_train=vectorize(X_train.values.astype('U'),tfidf_vect_fit)
model=RandomForestClassifier(n_jobs=-1,n_estimators=100)
model.fit(X_train,Y_train)
joblib.dump(model,"model.pkl",compress=3)
y_pred=model.predict(vectorize(X_test,tfidf_vect_fit))
print("Accuracy",round(accuracy_score(Y_test,y_pred),3))
print(time.time()-st)

['acceptable', 'accordance', 'account', 'act', 'action', 'activity', 'addendum', 'addition', 'additional', 'address', 'adj', 'adjustment', 'adverse', 'aed', 'age', 'agency', 'agent', 'agreed', 'agreement', 'agricultural', 'all', 'allowed', 'also', 'alteration', 'always', 'amount', 'analysis', 'another', 'any', 'anyone', 'anytown', 'applicable', 'applicant', 'application', 'apply', 'appraisal', 'appraised', 'appraiser', 'approach', 'appropriate', 'approval', 'apr', 'are', 'area', 'article', 'assessment', 'asset', 'assignment', 'assistance', 'association', 'assumption', 'attached', 'authorized', 'available', 'avery', 'balance', 'bank', 'bankruptcy', 'base', 'based', 'basement', 'basis', 'bath', 'best', 'bfe', 'borrower', 'boston', 'box', 'building', 'burden', 'bureau', 'business', 'calculation', 'cash', 'center', 'certificate', 'certification', 'certify', 'change', 'charge', 'check', 'choose', 'city', 'client', 'closing', 'code', 'collection', 'comment', 'common', 'community', 'company',

C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_store_unique_indices = np.zeros(y.shape, dtype=np.int)
C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to 

Accuracy 0.0
1.4638350009918213


C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


In [80]:
list(stopwords.stopwords())

AttributeError: 'WordListCorpusReader' object has no attribute 'stopwords'

In [91]:
ocr="""
Individual Condominium Unit Appraisal Report File # 
Freddie Mac Form 465 March 2005 Page 1 of 6 Fannie Mae Form 1073 March 2005 
The purpose of this summary appraisal report is to provide the lender/client with an accurate, and adequately supported, opinion of the market value of the subject property. 
Property Address Unit # City State Zip Code 
Borrower Owner of Public Record County 
Legal Description 
Assessor’s Parcel # Tax Year R.E. Taxes $ 
Project Name Phase # Map Reference Census Tract 
Occupant Owner Tenant Vacant Special Assessments $ HOA $ per year per month 
Property Rights Appraised Fee Simple Leasehold Other (describe) 
Assignment Type Purchase Transaction Refinance Transaction Other (describe) 
Lender/Client Address 
Is the subject property currently offered for sale or has it been offered for sale in the twelve months prior to the effective date of this appraisal? Yes No 
Report data source(s) used, offering price(s), and date(s). 
S 
U 
B 
J 
E 
C 
T
I did did not analyze the contract for sale for the subject purchase transaction. Explain the results of the analysis of the contract for sale or why the analysis was not 
performed. 
Contract Price $ Date of Contract Is the property seller the owner of public record? Yes No Data Source(s) 
Is there any financial assistance (loan charges, sale concessions, gift or downpayment assistance, etc.) to be paid by any party on behalf of the borrower? Yes No 
If Yes, report the total dollar amount and describe the items to be paid. 
C 
O
N 
T 
R 
A 
C 
T
Note: Race and the racial composition of the neighborhood are not appraisal factors. 
Neighborhood Characteristics Condominium Unit Housing Trends Condominium Housing Present Land Use % 
Location Urban Suburban Rural Property Values Increasing Stable Declining PRICE AGE One-Unit %
Built-Up Over 75% 25–75% Under 25% Demand/Supply Shortage In Balance Over Supply $ (000) (yrs) 2-4 Unit %
Growth Rapid Stable Slow Marketing Time Under 3 mths 3–6 mths Over 6 mths Low Multi-Family %
Neighborhood Boundaries High Commercial %
Pred. Other %
Neighborhood Description 
Market Conditions (including support for the above conclusions) 
N 
E 
I 
G
H 
B 
O
R 
H 
O
O
D
Topography Size Density View 
Specific Zoning Classification Zoning Description 
Zoning Compliance Legal Legal Nonconforming – Do the zoning regulations permit rebuilding to current density? Yes No 
 No Zoning Illegal (describe) 
Is the highest and best use of the subject property as improved (or as proposed per plans and specifications) the present use? Yes No If No, describe 
Utilities Public Other (describe) Public Other (describe) Off-site Improvements—Type Public Private 
Electricity Water Street 
Gas Sanitary Sewer Alley 
FEMA Special Flood Hazard Area Yes No FEMA Flood Zone FEMA Map # FEMA Map Date 
Are the utilities and off-site improvements typical for the market area? Yes No If No, describe 
Are there any adverse site conditions or external factors (easements, encroachments, environmental conditions, land uses, etc.)? Yes No If Yes, describe 
P 
R 
O
J 
E 
C 
T 
S 
I 
T 
E
Data source(s) for project information 
Project Description Detached Row or Townhouse Garden Mid-Rise High-Rise Other (describe) 
General Description General Description Subject Phase If Project Completed If Project Incomplete 
# of Stories Exterior Walls # of Units # of Phases # of Planned Phases 
# of Elevators Roof Surface # of Units Completed # of Units # o f Planned Units 
Existing Proposed Total # Parking # of Units For Sale # of Units for Sale # of Units for Sale 
 Under Construction Ratio (spaces/units) # of Units Sold # of Units Sold # of Units Sold 
Year Built Type # of Units Rented # of Units Rented # of Units Rented 
Effective Age Guest Parking # of Owner Occupied Units # of Owner Occupied Units # of Owner Occupied Units 
Project Primary Occupancy Principle Residence Second Home or Recreational Tenant 
Is the developer/builder in control of the Homeowners’ Association (HOA)? Yes No 
Management Group – Homeowners’ Association Developer Management Agent – Provide name of management company. 
Does any single entity (the same individual, investor group, corporation, etc.) own more than 10% of the total units in the project? Yes No If Yes, describe 
Was the project created by the conversion of an existing building(s) into a condominium? Yes No If Yes, describe the original use and the date of conversion. 
Are the units, common elements, and recreation facilities complete (including any planned rehabilitation for a condominium conversion)? Yes No If No, describe 
Is there any commercial space in the project? Yes No If Yes, describe and indicate the overall percentage of the commercial space. 
P 
R 
O
J 
E 
C 
T 
I 
N 
F 
O
R 
M
A 
T 
I 
O
N 
 Individual Condominium Unit Appraisal Report File # 
Freddie Mac Form 465 March 2005 Page 2 of 6 Fannie Mae Form 1073 March 2005 
Describe the condition of the project and quality of construction. 
Describe the common elements and recreational facilities. 
Are any common elements leased to or by the Homeowners’ Association? Yes No If Yes, describe the rental terms and options. 
Is the project subject to ground rent? Yes No If Yes, $ per year (describe terms and conditions) 
Are the parking facilities adequate for the project size and type? Yes No If No, describe and comment on the effect on value and marketability. 
P 
R 
O
J 
E 
C 
T 
I 
N 
F 
O
R 
M
A 
T 
I 
O
N
I did did not analyze the condominium project budget for the current year. Explain the results of the analysis of the budget (adequacy of fees, reserves, etc.), or why 
the analysis was not performed. 
Are there any other fees (other than regular HOA charges) for the use of the project facilities? Yes No If Yes, report the charges and describe. 
Compared to other competitive projects of similar quality and design, the subject unit charge appears High Average Low If High or Low, describe 
Are there any special or unusual characteristics of the project (based on the condominium documents, HOA meetings, or other information) known to the appraiser? 
 Yes No If Yes, describe and explain the effect on value and marketability. 
P 
R 
O
J 
E 
C 
T 
A 
N 
A 
L 
Y 
S 
I 
S
Unit Charge $ per month X 12 = $ per year Annual assessment charge per year per square feet of gross living area = $ 
Utilities included in the unit monthly assessment None Heat Air Conditioning Electricity Gas Water Sewer Cable Other (describe) 
General Description Interior materials/condition Amenities Appliances Car Storage 
Floor # Floors Fireplace(s) # Refrigerator None 
# of Levels Walls Woodstove(s) # Range/Oven Garage Covered Open 
Heating Type Fuel Trim/Finish Deck/Patio Disp Microwave # of Cars 
 Central AC Individual AC Bath Wainscot Porch/Balcony Dishwasher Assigned Owned 
 Other (describe) Doors Other Washer/Dryer Parking Space # 
Finished area above grade contains: Rooms Bedrooms Bath(s) Square Feet of Gross Living Area Above Grade 
Are the heating and cooling for the individual units separately metered? Yes No If No, describe and comment on compatibility to other projects in the market area. 
Additional features (special energy efficient items, etc.) 
Describe the condition of the property (including needed repairs, deterioration, renovations, remodeling, etc.). 
Are there any physical deficiencies or adverse conditions that affect the livability, soundness, or structural integrity of the property? Yes No If Yes, describe 
Does the property generally conform to the neighborhood (functional utility, style, condition, use, construction, etc.)? Yes No If No, describe 
U 
N 
I 
T 
D 
E 
S 
C 
R 
I 
P 
T 
I 
O
N
I did did not research the sale or transfer history of the subject property and comparable sales. If not, explain 
My research did did not reveal any prior sales or transfers of the subject property for the three years prior to the effective date of this appraisal. 
Data source(s) 
My research did did not reveal any prior sales or transfers of the comparable sales for the year prior to the date of sale of the comparable sale. 
Data source(s) 
Report the results of the research and analysis of the prior sale or transfer history of the subject property and comparable sales (report additional prior sales on page 3). 
ITEM SUBJECT COMPARABLE SALE # 1 COMPARABLE SALE # 2 COMPARABLE SALE # 3 
Date of Prior Sale/Transfer 
Price of Prior Sale/Transfer 
Data Source(s) 
Effective Date of Data Source(s) 
Analysis of prior sale or transfer history of the subject property and comparable sales. 
P 
R 
I 
O
R 
S 
A 
L 
E 
H 
I 
S 
T 
O
R 
Y
 Individual Condominium Unit Appraisal Report File # 
Freddie Mac Form 465 March 2005 Page 3 of 6 Fannie Mae Form 1073 March 2005 
There are comparable properties currently offered for sale in the subject neighborhood ranging in price from $ to $ . 
There are comparable sales in the subject neighborhood within the past twelve months ranging in sale price from $ to $ . 
FEATURE SUBJECT COMPARABLE SALE # 1 COMPARABLE SALE # 2 COMPARABLE SALE # 3 
Address and 
Unit # 
Project Name and 
Phase 
Proximity to Subject 
Sale Price $ $ $ $ 
Sale Price/Gross Liv. Area $ sq. ft. $ sq. ft. $ sq. ft. $ sq. ft. 
Data Source(s) 
Verification Source(s) 
VALUE ADJUSTMENTS DESCRIPTION DESCRIPTION +(-) $ Adjustment DESCRIPTION +(-) $ Adjustment DESCRIPTION +(-) $ Adjustment 
Sale or Financing 
Concessions 
Date of Sale/Time 
Location 
Leasehold/Fee Simple 
HOA Mo. Assessment 
Common Elements 
and Rec. Facilities 
Floor Location 
View 
Design (Style) 
Quality of Construction 
Actual Age 
Condition 
Above Grade Total Bdrms. Baths Total Bdrms. Baths Total Bdrms. Baths Total Bdrms. Baths 
Room Count 
Gross Living Area sq. ft. sq. ft. sq. ft. sq. ft. 
Basement & Finished 
Rooms Below Grade 
Functional Utility 
Heating/Cooling 
Energy Efficient Items 
Garage/Carport
Porch/Patio/Deck 
Net Adjustment (Total) + - $ + - $ + - $ 
Adjusted Sale Price 
of Comparables 
Net Adj. % 
Gross Adj. % $ 
Net Adj. % 
Gross Adj. % $ 
Net Adj. % 
Gross Adj. % $ 
Summary of Sales Comparison Approach 
S
A
L 
E
S
C
O
M
P
A
R
I 
S
O
N
A
P
P
R
O
A
C
H
Indicated Value by Sales Comparison Approach $ 
INCOME APPROACH TO VALUE (not required by Fannie Mae)
Estimated Monthly Market Rent $ X Gross Rent Multiplier = $ Indicated Value by Income Approach 
Summary of Income Approach (including support for market rent and GRM) 
I 
N
C
O
M
E
Indicated Value by: Sales Comparison Approach $ Income Approach (if developed) $ 
This appraisal is made “as is”, subject to completion per plans and specifications on the basis of a hypothetical condition that the improvements have been 
completed, subject to the following repairs or alterations on the basis of a hypothetical condition that the repairs or alterations have been completed, or subject to 
the following required inspection based on the extraordinary assumption that the condition or deficiency does not require alteration or repair: 
R
E
C
O
N
C
I 
L 
I 
A
T 
I 
O
N
Based on a complete visual inspection of the interior and exterior areas of the subject property, defined scope of work, statement of assumptions and limiting 
conditions, and appraiser’s certification, my (our) opinion of the market value, as defined, of the real property that is the subject of this report is 
$ , as of , which is the date of inspection and the effective date of this appraisal.
 Individual Condominium Unit Appraisal Report File # 
Freddie Mac Form 465 March 2005 Page 4 of 6 Fannie Mae Form 1073 March 2005 
This report form is designed to report an appraisal of a unit in a condominium project or a condominium unit in a planned 
unit development (PUD). This report form is not designed to report an appraisal of a manufactured home or a unit in a 
cooperative project. 
This appraisal report is subject to the following scope of work, intended use, intended user, definition of market value, 
statement of assumptions and limiting conditions, and certifications. Modifications, additions, or deletions to the intended 
use, intended user, definition of market value, or assumptions and limiting conditions are not permitted. The appraiser may 
expand the scope of work to include any additional research or analysis necessary based on the complexity of this appraisal 
assignment. Modifications or deletions to the certifications are also not permitted. However, additional certifications that do
not constitute material alterations to this appraisal report, such as those required by law or those related to the appraiser’s
continuing education or membership in an appraisal organization, are permitted. 
SCOPE OF WORK: The scope of work for this appraisal is defined by the complexity of this appraisal assignment and the 
reporting requirements of this appraisal report form, including the following definition of market value, statement of 
assumptions and limiting conditions, and certifications. The appraiser must, at a minimum: (1) perform a complete visual 
inspection of the interior and exterior areas of the subject unit, (2) inspect and analyze the condominium project, (3) inspect
the neighborhood, (4) inspect each of the comparable sales from at least the street, (5) research, verify, and analyze data 
from reliable public and/or private sources, and (6) report his or her analysis, opinions, and conclusions in this appraisal 
report. 
INTENDED USE: The intended use of this appraisal report is for the lender/client to evaluate the property that is the 
subject of this appraisal for a mortgage finance transaction. 
INTENDED USER: The intended user of this appraisal report is the lender/client. 
MARKET VALUE: The most probable price which a property should bring in a competitive and open market under all 
conditions requisite to a fair sale, the buyer and seller, each acting prudently, knowledgeably and assuming the price is not 
affected by undue stimulus. Implicit in this definition is the consummation of a sale as of a specified date and the passing of
title from seller to buyer under conditions whereby: (1) buyer and seller are typically motivated; (2) both parties are well 
informed or well advised, and each acting in what he or she considers his or her own best interest; (3) a reasonable time is 
allowed for exposure in the open market; (4) payment is made in terms of cash in U. S. dollars or in terms of financial 
arrangements comparable thereto; and (5) the price represents the normal consideration for the property sold unaffected by 
special or creative financing or sales concessions* granted by anyone associated with the sale. 
*Adjustments to the comparables must be made for special or creative financing or sales concessions. No adjustments are 
necessary for those costs which are normally paid by sellers as a result of tradition or law in a market area; these costs are 
readily identifiable since the seller pays these costs in virtually all sales transactions. Special or creative financing 
adjustments can be made to the comparable property by comparisons to financing terms offered by a third party institutional 
lender that is not already involved in the property or transaction. Any adjustment should not be calculated on a mechanical 
dollar for dollar cost of the financing or concession but the dollar amount of any adjustment should approximate the market’s 
reaction to the financing or concessions based on the appraiser’s judgment. 
STATEMENT OF ASSUMPTIONS AND LIMITING CONDITIONS: The appraiser’s certification in this report is 
subject to the following assumptions and limiting conditions: 
1. The appraiser will not be responsible for matters of a legal nature that affect either the property being appraised or the 
title to it, except for information that he or she became aware of during the research involved in performing this appraisal. 
The appraiser assumes that the title is good and marketable and will not render any opinions about the title. 
2. The appraiser has provided a sketch in this appraisal report to show the approximate dimensions of the improvements. 
The sketch is included only to assist the reader in visualizing the property and understanding the appraiser’s determination 
of its size. 
3. The appraiser has examined the available flood maps that are provided by the Federal Emergency Management Agency 
(or other data sources) and has noted in this appraisal report whether any portion of the subject site is located in an 
identified Special Flood Hazard Area. Because the appraiser is not a surveyor, he or she makes no guarantees, express or 
implied, regarding this determination. 
4. The appraiser will not give testimony or appear in court because he or she made an appraisal of the property in question, 
unless specific arrangements to do so have been made beforehand, or as otherwise required by law. 
5. The appraiser has noted in this appraisal report any adverse conditions (such as needed repairs, deterioration, the 
presence of hazardous wastes, toxic substances, etc.) observed during the inspection of the subject property or that he or 
she became aware of during the research involved in performing this appraisal. Unless otherwise stated in this appraisal 
report, the appraiser has no knowledge of any hidden or unapparent physical deficiencies or adverse conditions of the 
property (such as, but not limited to, needed repairs, deterioration, the presence of hazardous wastes, toxic substances, 
adverse environmental conditions, etc.) that would make the property less valuable, and has assumed that there are no 
such conditions and makes no guarantees or warranties, express or implied. The appraiser will not be responsible for any 
such conditions that do exist or for any engineering or testing that might be required to discover whether such conditions 
exist. Because the appraiser is not an expert in the field of environmental hazards, this appraisal report must not be 
considered as an environmental assessment of the property. 
6. The appraiser has based his or her appraisal report and valuation conclusion for an appraisal that is subject to 
satisfactory completion, repairs, or alterations on the assumption that the completion, repairs, or alterations of the subject 
property will be performed in a professional manner."""

In [92]:
import time,os,sys,re
import numpy as np
import pandas as pd
import pickle
import nltk
st=time.time()
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer,word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score
import joblib
stopwords=stopwords.words('english')
def clean(text):
    word_lem=WordNetLemmatizer()
    tokens=word_tokenize(text)
    lower=[word.lower() for word in tokens if len(word)>2 and word.isalpha() and word not in stopwords]
    lemmatized_text=[word_lem.lemmatize(word) for word in lower]
    return lemmatized_text
def vectorize(data,tfidf_vect_fit):
    x_tfidf=tfidf_vect_fit.transform(data)
    x_tfidf_df=pd.DataFrame(x_tfidf.todense(),columns=tfidf_vect_fit.get_feature_names())
    #print(tfidf_vect_fit.get_feature_names())
    return x_tfidf_df
import joblib
model=joblib.load("newmodel.pkl")
tfidf_vect_fit=joblib.load("newtfidf.pkl")
ocr_data=ocr
df2=pd.DataFrame({"data":[ocr_data]},dtype=str)
pred_value=model.predict(vectorize(df2["data"].values.astype('U'),tfidf_vect_fit))
pred_value[0]

C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


'1073'

In [93]:
len(df["data"])
a=[]

In [94]:
a=[]
for i in range(len(df["data"])):
    ocr_data=df["data"][i]
    df2=pd.DataFrame({"data":[ocr_data]},dtype=str)
    pred_value=model.predict(vectorize(df2["data"].values.astype('U'),tfidf_vect_fit))
    a.append(pred_value[0])

C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the releas

C:\Users\Yokesh\Downloads\python\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


In [95]:
a

['1003',
 '1008',
 '1073',
 'closing disclosure',
 'credit report',
 'credit report',
 'detail aprisal',
 'detail aprisal',
 'detail aprisal',
 'detail aprisal',
 'detail aprisal',
 'detail aprisal',
 'flood cert',
 'flood cert',
 'tax responsibility letter']

In [96]:
Y_train

4          credit report
9         detail aprisal
7         detail aprisal
3     closing disclosure
2                   1073
8         detail aprisal
11        detail aprisal
12            flood cert
0                   1003
6         detail aprisal
10        detail aprisal
1                   1008
5          credit report
13            flood cert
Name: label, dtype: object

In [97]:
Y_test

14    tax responsibility letter
Name: label, dtype: object